# Academia–Practice Interaction Mapping Using NLP

**Notebook 03: Exploratory Data Analysis (EDA)**

**Author:** Kamila Lewandowska  
**Project Status:** *In Progress*  
**Last Updated:** April 2025  

---

### Notebook Overview

This notebook assumes that data cleaning and NER extraction have already been completed.  
It performs comparison, overlap analysis, and exploratory stats to understand similarities and differences in outputs between the two models.

---

### File Overview

**Focus:** Compare named entities of type `ORG` extracted using two NLP models:  
- Stanza (rule-based, Polish language)
- XLM-RoBERTa (transformer-based multilingual model from HuggingFace)


---


# EDA: Compare entities from both models

In [19]:
import pandas as pd
from ast import literal_eval
from collections import Counter

# Load previously extracted NER results
stanza_pl = pd.read_csv("../output/ner_stanza_pl.csv")
davlan_pl = pd.read_csv("../output/ner_davlan_pl.csv")

In [4]:
# Create a dataframe for comparison 

ner_pl_comp = pd.merge(stanza_pl, davlan_pl, on = ["Text", "ICS_ID"], how="inner")


In [5]:
# Change the order of columns

ner_pl_comp = ner_pl_comp[['ICS_ID', 'Text', 'ORG_Entities_stanza', 'ORG_Entities_xlm']]

In [6]:
ner_pl_comp.columns

Index(['ICS_ID', 'Text', 'ORG_Entities_stanza', 'ORG_Entities_xlm'], dtype='object')

In [7]:
ner_pl_comp.head()

,ICS_ID,Text,ORG_Entities_stanza,ORG_Entities_xlm
0,00153fbd-82f7-48c4-b5bd-e830bc390244,Badania skupiające się na szczegółowej analizi...,['Komitetu Nauk Weterynaryjnych i Rozrodu Zwie...,['Komitetu Nauk Weterynaryjnych i Rozrodu Zwie...
1,002768f1-8b96-4e0f-bcc8-192eb0594e60,"Birdwatching, czyli obserwacje w terenie ptakó...","['Królewskie Towarzystwo Ochrony Ptaków', 'Fac...","['Królewskie Towarzystwo Ochrony Ptaków', 'Zak..."
2,00500483-f00c-4410-b6f7-8650a003125f,Efektywny transfer wiedzy jest podstawowym czy...,"['MŚP', 'MŚP', 'ETW']",[]
3,006e7fef-2083-426d-9c1b-1affd27b939e,Ważnym obszarem działalności naukowej WSPiA je...,"['WSPiA', 'AP', 'WSPiA', '4 Zespoły', 'AP', 'A...","['WSPiA', 'WSPiA']"
4,00901439-d91a-48e0-903a-26a4253c3a0c,Znaczna część europejskiego dziedzictwa archeo...,"['Interreg Central Europe', 'Archaeological He...","['Inter', 'Archaeological Heritage Office of S..."


In [8]:
# Checking the data type

print(type(ner_pl_comp.iloc[0, 2]))

<class 'str'>


In [11]:
# Converting columns with entity names into lists

ner_pl_comp[["ORG_Entities_stanza", "ORG_Entities_xlm"]] = ner_pl_comp[["ORG_Entities_stanza", "ORG_Entities_xlm"]].apply(
    lambda col: col.map(literal_eval))


In [12]:
# Create a column with entities detected by both models

ner_pl_comp["Common_entities"] = ner_pl_comp.apply(
    lambda row: list(set(row["ORG_Entities_stanza"]) & set(row["ORG_Entities_xlm"])), axis=1
                    )

In [13]:
ner_pl_comp.head()

,ICS_ID,Text,ORG_Entities_stanza,ORG_Entities_xlm,Common_entities
0,00153fbd-82f7-48c4-b5bd-e830bc390244,Badania skupiające się na szczegółowej analizi...,[Komitetu Nauk Weterynaryjnych i Rozrodu Zwier...,[Komitetu Nauk Weterynaryjnych i Rozrodu Zwier...,"[ELSEVIER, Komitetu Nauk Weterynaryjnych i Roz..."
1,002768f1-8b96-4e0f-bcc8-192eb0594e60,"Birdwatching, czyli obserwacje w terenie ptakó...","[Królewskie Towarzystwo Ochrony Ptaków, Facebo...","[Królewskie Towarzystwo Ochrony Ptaków, Zakład...","[Królewskie Towarzystwo Ochrony Ptaków, UAM]"
2,00500483-f00c-4410-b6f7-8650a003125f,Efektywny transfer wiedzy jest podstawowym czy...,"[MŚP, MŚP, ETW]",[],[]
3,006e7fef-2083-426d-9c1b-1affd27b939e,Ważnym obszarem działalności naukowej WSPiA je...,"[WSPiA, AP, WSPiA, 4 Zespoły, AP, AP, ZK, ZK, ...","[WSPiA, WSPiA]",[WSPiA]
4,00901439-d91a-48e0-903a-26a4253c3a0c,Znaczna część europejskiego dziedzictwa archeo...,"[Interreg Central Europe, Archaeological Herit...","[Inter, Archaeological Heritage Office of Saxo...","[Cultural Heritage Department, Archaeological ..."


In [14]:
# Create stanza and davlan columns with unique entities per row (entities can repeat across rows but not in the same row/ ics)

ner_pl_comp["Stanza_unique_row"] = ner_pl_comp["ORG_Entities_stanza"].apply(lambda row: list(set(row)))
ner_pl_comp["Davlan_unique_row"] = ner_pl_comp["ORG_Entities_xlm"].apply(lambda row: list(set(row)))


In [15]:
ner_pl_comp.head()

,ICS_ID,Text,ORG_Entities_stanza,ORG_Entities_xlm,Common_entities,Stanza_unique_row,Davlan_unique_row
0,00153fbd-82f7-48c4-b5bd-e830bc390244,Badania skupiające się na szczegółowej analizi...,[Komitetu Nauk Weterynaryjnych i Rozrodu Zwier...,[Komitetu Nauk Weterynaryjnych i Rozrodu Zwier...,"[ELSEVIER, Komitetu Nauk Weterynaryjnych i Roz...","[ELSEVIER, Katedry, Komitetu Nauk Weterynaryjn...","[ELSEVIER, Komitetu Nauk Weterynaryjnych i Roz..."
1,002768f1-8b96-4e0f-bcc8-192eb0594e60,"Birdwatching, czyli obserwacje w terenie ptakó...","[Królewskie Towarzystwo Ochrony Ptaków, Facebo...","[Królewskie Towarzystwo Ochrony Ptaków, Zakład...","[Królewskie Towarzystwo Ochrony Ptaków, UAM]","[Królewskie Towarzystwo Ochrony Ptaków, Region...","[Królewskie Towarzystwo Ochrony Ptaków, UAM, „..."
2,00500483-f00c-4410-b6f7-8650a003125f,Efektywny transfer wiedzy jest podstawowym czy...,"[MŚP, MŚP, ETW]",[],[],"[MŚP, ETW]",[]
3,006e7fef-2083-426d-9c1b-1affd27b939e,Ważnym obszarem działalności naukowej WSPiA je...,"[WSPiA, AP, WSPiA, 4 Zespoły, AP, AP, ZK, ZK, ...","[WSPiA, WSPiA]",[WSPiA],"[4 Zespoły, AP, WSPiA, ZK]",[WSPiA]
4,00901439-d91a-48e0-903a-26a4253c3a0c,Znaczna część europejskiego dziedzictwa archeo...,"[Interreg Central Europe, Archaeological Herit...","[Inter, Archaeological Heritage Office of Saxo...","[Cultural Heritage Department, Archaeological ...","[Cultural Heritage Department, CMAS, UE, Urzęd...","[Cultural Heritage Department, Inter, Archaeol..."


In [16]:
# Calculate entities in each column:
# Stanza_all = all entities extracted by the model (with duplicates within and across rows)
# Stanza_unique_row = no duplicates within rows
# Davlan_all_entities = all entities extracted by the model (with duplicates within and across rows)
# Davlan_unique_row = = no duplicates within rows
# Common_sum = entities extracted both by Stanza and Davlan (no duplicates)

stanza_all = sum(ner_pl_comp["ORG_Entities_stanza"].apply(lambda row: len(row)))
stanza_unique_row = sum(ner_pl_comp["Stanza_unique_row"].apply(lambda row: len(row)))
davlan_sum = sum(ner_pl_comp["ORG_Entities_xlm"].apply(lambda row: len(row)))
davlan_unique_row = sum(ner_pl_comp["Davlan_unique_row"].apply(lambda row: len(row)))
common_sum = sum(ner_pl_comp["Common_entities"].apply(lambda row: len(row)))

print(f'Stanza all entities: {stanza_all}')
print(f'Stanza unique per row: {stanza_unique_row}')
print(f'Davlan all entities: {davlan_sum}')
print(f'Davlan unique per row: {davlan_unique_row}')
print(f'Common entities: {common_sum}')

Stanza all entities: 35642
Stanza unique per row: 25714
Davlan all entities: 17554
Davlan unique per row: 14080
Common entities: 8803


In [17]:
# Calculate the overlap between Stanza and Davlan (Jaccard similarity percentage): 
# Jaccard Similarity = (Number of common elements) / (Number of unique elements in both sets)

overlap_percent = 8803 / (25714 + 14080 - 8803) * 100
print(overlap_percent)


28.405020812493948


In [20]:
# Explore frequencies of entities

stanza_flat_list = [entity for row in ner_pl_comp["Stanza_unique_row"] for entity in row]
stanza_freq = Counter(stanza_flat_list)
print("Stanza:")
print(stanza_freq.most_common(50))

davlan_flat_list = [entity for row in ner_pl_comp["Davlan_unique_row"] for entity in row]
davlan_freq = Counter(davlan_flat_list)
print("Davlan:")
print(davlan_freq.most_common(50))

both_flat_list = [entity for row in ner_pl_comp["Common_entities"] for entity in row]
both_freq = Counter(both_flat_list)
print("Stanza & Davlan:")
print(both_freq.most_common(50))

Stanza:
[('UE', 341), ('Unii Europejskiej', 167), ('Komisji Europejskiej', 105), ('Instytutu', 102), ('NCBiR', 61), ('Instytut', 60), ('ONZ', 57), ('UJ', 51), ('UW', 47), ('UNESCO', 47), ('unijnych', 46), ('Zespołu', 45), ('NFZ', 44), ('Uczelni', 43), ('GUS', 42), ('WHO', 40), ('Instytucie', 39), ('Rady Ministrów', 38), ('Facebook', 37), ('UMK', 37), ('KE', 36), ('Rady', 36), ('Parlamentu Europejskiego', 34), ('NATO', 32), ('UAM', 31), ('Komisję Europejską', 31), ('NCN', 30), ('UWr', 29), ('AGH', 29), ('OZE', 29), ('MŚP', 27), ('Komisja Europejska', 27), ('Muzeum', 27), ('UP', 27), ('OECD', 26), ('TVP', 26), ('PW', 26), ('UŁ', 26), ('B+R', 26), ('Polskiego Radia', 25), ('Wydziału', 25), ('SGGW', 25), ('PG', 25), ('Polonii', 24), ('NGO', 24), ('Policji', 24), ('MRiRW', 23), ('europejskiej', 23), ('Ministerstwa Zdrowia', 23), ('EU', 23)]
Davlan:
[('', 190), ('UE', 171), ('Instytut', 100), ('Unii Europejskiej', 64), ('Komisji Europejskiej', 56), ('In', 51), ('UJ', 35), ('WHO', 34), ('UW',

In [21]:
# Create a list of unique entities per model

unique_stanza = list(set(stanza_flat_list))
unique_davlan = list(set(davlan_flat_list))
unique_common = list(set(both_flat_list))

unique_stanza_only = list(set(stanza_flat_list) - set(davlan_flat_list))
unique_davlan_only = list(set(davlan_flat_list) - set(stanza_flat_list))

unique_stanza_count = len(unique_stanza)
unique_davlan_count = len(unique_davlan)
unique_common_count = len(unique_common)
unique_stanza_only_count = len(unique_stanza_only)
unique_davlan_only_count = len(unique_davlan_only)

print(f'Unique entities Stanza: {unique_stanza_count}')
print(f'Unique entities Davlan: {unique_davlan_count}')
print(f'Unique common entities: {unique_common_count}')
print(f'Unique Stanza only: {unique_stanza_only_count}')
print(f'Unique Davlan only: {unique_davlan_only_count}')

Unique entities Stanza: 17988
Unique entities Davlan: 9751
Unique common entities: 6111
Unique Stanza only: 11720
Unique Davlan only: 3483


In [23]:
# Save common entities to a DataFrame
df_common = pd.DataFrame({"ORG_Entity": unique_common})
df_common.to_csv("../output/common_org_entities.csv", index=False)

print("Saved common entities to /output/common_org_entities.csv")

Saved common entities to /output/common_org_entities.csv
